### Notes 

T5 Paper: https://arxiv.org/pdf/1910.10683.pdf

T5 Tokenizer: https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_t5.py

Important Tasks: https://docs.google.com/document/d/1weIZM6QTlnitpPQmpg-WeV2RW70TnYmDuogBQPr5mB0/edit

In [1]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
import pandas as pd
import numpy as np
import pickle
import os, io, glob

import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchtext.datasets.translation import TranslationDataset
from torchtext.data import Field, BucketIterator

import sentencepiece as spm

from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config


from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab

from sklearn.model_selection import train_test_split


In [2]:
ls data/AD_NMT-master

LAV-MSA-2-both.pkl        Magribi_MSA-train.pkl     english-arabic-test.pkl
LAV-MSA-2-test.pkl        README.md                 lav_formatted_.txt
LAV-MSA-2-train.pkl       en_formatted_.txt         mag_formatted_.txt
Magribi_MSA-both.pkl      english-Arabic-both.pkl   msa_formatted_.txt
Magribi_MSA-test.pkl      english-Arabic-train.pkl


In [62]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    en_msa_pre_train_ds = pickle.load(handle) 
    
with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    en_msa_test_ds = pickle.load(handle) 
    
with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    lav_msa_pre_train_ds = pickle.load(handle)
    
with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    lav_msa_test_ds = pickle.load(handle)    
    
with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA = pickle.load(handle) 

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_English_MSA = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA = pickle.load(handle) 
    

In [63]:
print(en_msa_pre_train_ds[0:5])
print(en_msa_test_ds[0:5])

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[["Where's your money?", 'اين مالك؟'], ['Be prepared', 'كن مستعدا'], ["I figured you'd be impressed", 'توقعت انك ستنبهر'], ['May I come in?', 'هل بامكاني الدخول؟'], ['Read through the article', 'اقرا المقالة']]


## Prepare Datasets

example: https://iwslt2010.fbk.eu/node/32/

We need to take our training and test sets from the pkl files and create new .txt files that are formatted so that the standard torchtext Dataset class can read them

Data format:
each line consists of three fields divided by the character '\'
sentences consisting of words divided by single spaces
format: <SENTENCE_ID>\<PARAPHRASE_ID>\<TEXT>
Field_1: sentence ID
Field_2: paraphrase ID
Field_3: MT develop sentence / reference translation
Text input example:
DEV_001\01\This is the first develop sentence.
DEV_002\01\This is the second develop sentence.
Reference translation example:
DEV_001\01\1st reference translation for 1st input
DEV_001\02\2nd reference translation for 1st input
...
DEV_002\01\1st reference translation for 2nd input
DEV_002\02\2nd reference translation for 2nd input
...
Languages:
Arabic-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
IWSLT07 testset: 489 sentences, 6 reference translations
IWSLT08 testset: 507 sentences, 16 reference translations
French-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
Turkish-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations


In [5]:
ls data/AD_NMT-master/

LAV-MSA-2-both.pkl        Magribi_MSA-train.pkl     english-arabic-test.pkl
LAV-MSA-2-test.pkl        README.md                 lav_formatted_.txt
LAV-MSA-2-train.pkl       en_formatted_.txt         mag_formatted_.txt
Magribi_MSA-both.pkl      english-Arabic-both.pkl   msa_formatted_.txt
Magribi_MSA-test.pkl      english-Arabic-train.pkl


In [105]:
ls data/train

en_formatted_.txt


In [94]:
rm data/train/*

In [119]:
file_path = 'data/'

def pytorch_format(ds, src='en', trg='msa',datatype=''):
    src_formatted = src + '_formatted_' + '.txt'
    trg_formatted = trg + '_formatted_' + '.txt'
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as enpy, open(file_path + datatype + "/" + trg_formatted, 'wt') as arpy:
        for i, arr in enumerate(ds):
            print(arr[0])
            enpy.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[0] + '\n')
            arpy.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[1] + '\n')

In [123]:
#pytorch_format(data_LAV_MSA, 'lav', 'msa', '')
#pytorch_format(data_English_MSA, 'en', 'msa', '')
#pytorch_format(data_Magribi_MSA, 'mag', 'msa', '')

In [125]:
#splits the datatsets into train and validation sets 
en_msa_train_ds, en_msa_val_ds = train_test_split(en_msa_pre_train_ds, test_size=.2)
lav_msa_train_ds, lav_msa_val_ds = train_test_split(lav_msa_pre_train_ds, test_size=.2)

In [80]:
print(len(en_msa_train_ds))
print(len(en_msa_val_ds))

7200
1800


In [120]:
pytorch_format(en_msa_train_ds, 'en', 'msa', 'train')
pytorch_format(en_msa_val_ds, 'en', 'msa', 'validation')
pytorch_format(en_msa_test_ds, 'en', 'msa', 'test')

pytorch_format(lav_msa_train_ds, 'en', 'msa', 'train')
pytorch_format(lav_msa_val_ds, 'en', 'msa', 'validation')
pytorch_format(lav_msa_test_ds, 'en', 'msa', 'test')

We discussed the plan with him
Tom doesn't actually live within Boston city limits
I cannot thank you enough for all your kindness
She turned down my request
Tom knows that Mary loves him
I'm very thirsty
The women of France are beautiful
We don't know her
How will we pay our debts now?
Good morning It's time to wake up
May I take a photo?
An investigation is going on
I'm your father
I told you I don't know how to do that
I met him on the street the other day
Are you ready to go home now?
The gang was planning a robbery
I've just finished eating breakfast
Kuala Lumpur is worth visiting
I don't like him because he is sly as a fox
Tourists poured into Karuizawa during the summer vacation
He is such a bad person that everybody dislikes him
I'll have to think it over
Mary is Tom's secretary
You must give up smoking
He is busy doing something
Tom isn't going to do that
I'm not in the best of shape
It's well worth the trouble
I never for a moment imagined that I would win
It feels like a dre

Watch out the man has a gun
Let's ask her when she gets back home
Why didn't they say something?
Can you help me with my work?
Everybody is waiting for you
Why is autumn called "fall" in America?
You must be kidding!
Tom tried not to look at Mary
You don't have to listen to what he says
She has a son and two daughters
He is a man of wit
There are lots of animals in the park
I'm not thin
Were you there before?
Think about how you'd feel if that happened to you
I was happy then
Tom knows that you know
Cancer can be cured if discovered in time
I met Tom on my way to school
Please shut the door behind you
He lost his job
Don't use my pen
Would you play with me?
I have a cold
Is it possible that it's already eight o'clock?
I got on the wrong train
There are numerous universities in Kyoto
It's more difficult than you think
If you've got something to say just say it
I love French films
Would you please tell me your name again?
Police are investigating Tom
Can you help me to translate these se

She almost fainted
This is the case
What do you mean?
Can you speak French?
I'm glad to see you back
Books are my best friends
Everybody loves her
May I ask where are you going?
Chew your food well
I can't thank you enough
We must all take care to preserve our national heritage
I just don't want you to do something you'll be sorry for
Tom refused to talk to me
I knew that Tom would come
Why is that wrong?
I first used a computer about 40 years ago
My uncle is angry
Tom tried to kill us
This is an extremely important point
He has a car that has four doors
We have to start somewhere
My father has never been abroad
Not everybody can be a poet
He claimed he knew you well
Tom lost three million dollars last year
Tom and Mary exchanged smiles
Don't you play tennis?
I remember seeing him somewhere before
I'm OK
Why did you abandon me?
I think you're right
Thanks for your explanation
How often do you go abroad?
I gave him all the money I had
An accident has happened
Well let's go
He looks like

I came here yesterday
I'm at work now so I'll call you later
What's happened to the money?
I'm sure you're right
My father made me wash the car
Above all watch your diet
He is a fast speaker
There are a few shops next to my house
It appears to me that you are all mistaken
He believes whatever I say
His plan is dangerous!
There isn't any solution
Are you able to swim?
I came on foot
Just a moment please
He fell in love with her at first sight
What time did you arrive there?
Your problem is similar to mine
I regret missing the speech
In which folder did you save the file?
It is really nice of you
We learn by experience
I'll join you later
What is new?
Those books are mine
I love you
I don't want to work
Welcome home
I admit that I was careless
She is not tall
Do you remember us?
I stay in Osaka
I have a dog and a cat
There's a possibility of war
Just tell me what you want
I waited for Tom for a long time
I broke a glass
I miss you very much
Is your watch correct?
I tried not to look disa

Bring him in
Here it is
I've been thinking of you all day
Let's get started right away
You shouldn't judge a person by his appearance
Don't hang up yet please
The drug smuggler was arrested at the airport
No one ate the cake
Allow me to go
We can do nothing about it
It looks like an apple
It isn't here anymore
The nurse took his temperature
I wouldn't be calling if it weren't important
Because he's sick he can't come
I am thirsty
Isn't there something you want to say to me?
Tom got tired of reading and wanted to go outside and play
He has started to write a novel
You can use it anytime
Even children can understand it
She lives in New York
English is difficult isn't it?
I just wanted to hear your voice
I called Tom up
I already went to the butcher's shop this morning
You must show your passport
Could I have a glass of white wine?
I do hope so
We admire her for her bravery
The engine started again
I suggested to Tom the possibility that he work with Mary
Rats breed rapidly
Did you show i

...

عموما ما تلومي حالك كتير 
بسم الله ما شاء الله
ليش بيعطوه شو بده بقله شو بده و بيعملي يلي بدي ياه
بس لازمك تاخذي البنت عندها لتتاكد من القياس لانه في احتمال ما يناسبهاش
بتبطلي هالشغلة رح غير عالتجارة و هيك شغلات او لا
اه عندي في الجاكيت السوداء شوفي في الجاكيت السوداء
امتى نلتقي على السكايب 
تصدق يا اخوي هذا مفيد
اي كانوا حيوانات
وانا يوم بقول هدا رايي الشخصي
يا الهي
قالتلنا سالتني مرة مصرية ليش بيتجوا معجزين
لا انا صراحة لما الاقي مستقبل هان في بلدي حاضلني
العسل عنده دنين هذه ما بعرفها
شو بدي قلها بكرة
عم استنى رايكم 
وصيتي الك حبيبي انو ما تنساني 
صدقيني ما بعرف يا زينب
الف مبروك رضوى  الله يفرحك 
بيجي ليسكروه بئلو اسمع لا تسكر
بتعرف لك محمد لما بكون صايب ما بحب اطلع لبرا بخاف احكي مع العالم
اي
اه
وين بتقع المدرسة العليا للعلوم السياسية
بس دوريلي على عنوانة ماعطتني ياه البنت
برافو سليمة بالعقل
ما عملتش اشي اشياء بسيطة كتير و عملت فيهم اشي زاكي
يلا هيها بدت
هههه انا عازمتك على موضوعي الجديد 
سارة عطيني ياها هلق بس جبتها
ابويا ببيصلي في مسجد بلال
شوفت لما بدا الكل يطيب و بدا المرض يروح

نازل هلق من السلم
الاستاز ي ابني هو اتداين الف وبعدها الف وما رح يرجع اشي
جيب الدواء
احيانا بشعر بالتعب بمجرد ما ابذل مجهود صغير
احسنلها هديك اللي كانت عندي في الصيف هديك كنت اشمها من بعيد
الو
قلتك راحوا عالغابة و هما ما بيعرفوش كيف يصطادوا صاروا يهجموا على اللحامين و شحنات اللحم
شو رايك
بدك تحطيلي علامة
اي بالعجين
معلش حبيبيتي موضوعك حلو كتير كتير 
اكيد هاد نوع من التسويق
هذه القصة قديمة
تعاي عدي معنا امي
التعليم كمان يا بنيتي مش سهل بالمرة
يعني قوي
قلها مافي داعي خليها للصبح
متل ما قلتلك قبل هالمرة بالحلوة والمرة 
هو عندو شنطة شرايح
مع اني مش زملكاوية 
ما بعرف اذا مراد عالنت او عم يقرا
وبكتب صح وبدون ما حدا ينقلني 
ولا تصدي انو في تلطيشات ولا شي 
و شو بالنسبة للمحاسبة
اي
انا شخصيا ما بقى اتحمل شوف كل المظاهر
رجعتلك الدكتورة الوصفة
يا مختار هو بغلب البنت
بعدين سالت الثاني شو بتعنيلك مصر 
العطر التاني احسن منو
زي هدى مخلصة شهادة البكالوريوس في مراقبة المالية روعة كتير
اذا جيد وكمان ما كتبو بالشهادة مرفوعا تقديرو 
اذا اتطلع بعيسة بيحترق و بيدوب لانو بيضل يحوم حواليه مشان مايشوفو و هوي ع

عموما ما تلومي حالك كتير 
الشبكات
ولازم تتبع الاوامر بس متعه ما بعدها متعه 
ماشي و ايمت بترجعي
يمكن
طبعا كمان عشان يمنعو تتبع اثارك عالنت
يا مو شو رايك بحلو معمول من الشوكولا
و مرات تانية استخدمه للسيطرة على اخوي الاصغر اللي يستغل خفته و صغر حجمه و مراوغته السريعة و بنسى ان في على للسطح حبل مبربوط
ما عليكم الا ان تنشروها في كل المنتديات
سكرت الرسالة بسرعة و رجع فتحتا و بلشت اقراها من جديد
شبك نحنا عم نشوف فيلم
ما ضلوش عندي بالمرة
اه انا قادرة
قلتله حاقولك اشي لو حملت لابتوب و لفيت على الصيدليات بقدر اخدهم والا لا
كنت بفكر انه القانون ما بيحق الا للادبي
كتاب او كتابين  كل شيء مجانا 
ما في تمانة بالخط مو هيك امينة
الناس ما بيرحموش
لساتها عندك
و دراستك هي سلاحك
سبعة
مشوا شوي يقرا قران ترجع توئع
علجية جهزي الزنار
بعد هيك مافي شي
فعلا  انا اقترحت 10 ونص
ما بتابع نشرة الاخبار كتير و ما بعرف اذا طلع كتير او قليل
الجني جابلو الكشري وقعد ياكل فيه  وبس خلصو سالو شو طلبك التاني  الصعيدة واحد كشري كمان وحياة ابوك
اختفى كلش لا لا تطفيه خليه يكمل
بدك تشتري جزمة
من كذا يوم كانت فريال زردازي بتحكيلي ع

والله الموضوع حلو وشكرا الك وانا مش منزعج الحمدلله ولا من اشي
ما اتصلنا
لكن شو بيقصدوا بابتداء من
بس اذا غبتي تلات مرات بيحرموكي من الفحص و اذا غبتي مرة لازم تجيبي تقرير طبي تاني يوم ما في مزح
في هاليومين لقيت حالي كل ما بروح بلاقي حالي بحط راسي على المخدة
وانا من الناس الي بتموت اذا ما اكلو كنتالوب 
وين انت سليمة 
بختار الجزائر
يوم ما رحنا سوا
انشاء الله بخير
بخير و الحمد لله
مو موجودة كتير
اول مرة بشوف وشك هيك
شايفة ايش بلبس يا بنتي
لا انتي بتعرفي زوقي بالفساتين و بتعرفي طريقتي و بتعرفي شو بحب
لالا مو هو الاخير
نتنفس من وقت للتاني بس ونرجع مش حنهرب
المشفى فيه موظفين من كل الدول من اوروبا و افريقيا و كانت علاقتي فيون علاقة عمل
حتى انا
طيب ما في مشكلة
المهم انه ربنا ستر
راح ابلغهم مع السلامة
كل واحد بتلاقيه عندو شي اضافي مع الدراسة شي بيكرهو بالدراسة
مزبوط نحن منوصل قبل تبع الامن
صدقيني انو حالتو بتاسي
وناس دايما مش موجودين حتى لو كانو قريبين 
سمعي ئولي ل حورية و مريم و الزهرة انو معزومين
كتكوتة مشاغبة عاملة دوشة 
ايش قلتيلي ايش بدك تعملي في المستقبل ايش بدك تصيري
ايش مالها رجلك قديش ق

اشتقتولها
لان المطبوعات الها تاثير اقوى من التوزيع الالكتروني
وعندك فكرة عن الاعلام الالي او لا
اسمع كل شي مرتبط بالدين
حكالي عنها ابن عمك
رح جيب شي حلو للاكل
اه رائع هالشي
اذا ليلة سعيدة
ما ضل في فحوص شفهية
ع بالي لو نحكي بهالخصوص
اي دين هاد يلي بيسمح بالظلم
اي موضة يا سليمة بيتهيالك انهم ديك مبلول
راح يدور عالولد عند الخادمة فما لقاه عندا قام جن جنونو
اه بالتاكيد
ونستنى منك اكتر واكتر 
بس بدون شغل او اي شي و ما عندهن سكن
اه طلعت
اهل العريس و العروس حماتي و كنتي
هي القصة واقفة على الكي بورد
هاد هو
لما قلتيلي قوليلها في الجوال ضحكت كتير
من بعد ابداعك رح نبيع كبدة
بنصحكو تحطوا الذال
كل واحد بياخد اللي الله كاتبله ياه
انشاء الله بتعطل الفارة تبعتك متل ما كانت عم تقلك الختيارة مبارح
منيح ما عنا كاميرا هاد من حظي
مسا الخير
خليني مر شو ساويتلهن
و اسمعي بتقولك عقيلة اتصلي انت بس بحورية و بنتها
وين لقيتها
عن شو بتحب نحكي كمال اقترحلك شي موضوع
اه انا بتوجع كتير
انا رايحة عشان اخذلها شهادة مرضية
رجع زوجا من الشغل و سالا عن ابنو
هو طلب منهم يحضرو ع شكل تطبيقات
اكيد انك نزلت شغلة فيه
قلت لك بيخاف

In [121]:
ls data/train

en_formatted_.txt   msa_formatted_.txt


In [122]:
cat data/train/en_formatted_.txt

TRAIN_0000\01\يمكن تكون مسالة السلاح الكيمياوي مفتعلة عشان ياخدوها حجة زي ما صار في العراق
TRAIN_0001\01\بدها شوية معالجة
TRAIN_0002\01\احسبي فرق الساعات بين ادان العشي و ادان الصبح و قسمي المجموع ع تلاتة بتطلع معك النتيجة
TRAIN_0003\01\اي منيح هو
TRAIN_0004\01\و بتوفيق الله لقيت على غلافه عنوان ناشر الكتاب فراسلته عشان اطلب كمان فبعتولي اللي طلبته
TRAIN_0005\01\اقوله اذا قدر انه ياخذلي موعد عنده
TRAIN_0006\01\قصدي احترم حالك هاد كل شي
TRAIN_0007\01\لو اني مثلا الاقي وظيفة في الدولة و اشتغل هناك بقدر اني اخذ راتب من هان و من هناك
TRAIN_0008\01\لا بديش العب
TRAIN_0009\01\هلء رح يبلش ترديد كلمات اقرا حفاظ اكتوب و البقية
TRAIN_010\01\كانك عم تعلم حدا كيف بدو يعلم شي
TRAIN_011\01\ركزي تفكيرك و سيبي اللعب على جنب و خلي عنك هذه القصص و صفي ذهنك
TRAIN_012\01\عنجد
TRAIN_013\01\مسكت زينو اليوم
TRAIN_014\01\وهو هيك
TRAIN_015\01\واحد متلو بيفكر بمصلحتو الشخصية قبل مصلحة الطلاب
TRAIN_016\01\حطيت ملف لاخد عقد
TRAIN_017\01\اي
TRAIN_018\01\مابعرف
TRAIN_019\01\مش حاتاخر فيه
TRAIN_020\01\بتعرفي انه اسا

TRAIN_4664\01\شو ��م تساوي بحياتك شو عم تدرسي
TRAIN_4665\01\بتعرف قربت اموت من خجلي قوم الدنيا اما اللي ما بتسمى صاروا يطلعوا بس قالوله ايش قالتلك
TRAIN_4666\01\اقرالي ايش كتبت
TRAIN_4667\01\قررت اجمع مهر لمرتي و كان الشغل ماشي منيح و مرتب و خاصة انو شغلي كان بمشفى عسكري
TRAIN_4668\01\ههههه 
TRAIN_4669\01\ضحكني من كم يوم قلي عطتني انسة الانكليزي ورقة الفحص فقلتلا معقول يا انسة حفظت كل دروسي و حاولت غش و تحركت بكل الاتجاهات و اخدت تمانة بالاخير
TRAIN_4670\01\اي هجرة 
TRAIN_4671\01\التعليم كمان يا بنيتي مو سهل ابدا
TRAIN_4672\01\باي باي
TRAIN_4673\01\ما تروحش سيب السيارة
TRAIN_4674\01\غي و دئي لباقي مشان يخبر كوكة تجيب معها خبز
TRAIN_4675\01\والطحين ما رخص
TRAIN_4676\01\لا تنسانا ابدا
TRAIN_4677\01\عندك تطبيق ولا شو
TRAIN_4678\01\منيح انك صائم
TRAIN_4679\01\ولا لحظه عمره ما نسيها
TRAIN_4680\01\واوعى تبعد من هون مشان ما تضيع 
TRAIN_4681\01\بينقال بيقدر انه يفتن اي واحد
TRAIN_4682\01\بعتتني مريم جبلها وثيقة
TRAIN_4683\01\قالتلي بيحب فريد كتير
TRAIN_4684\01\اذا شي استاذ ضرب زميلي باضرب
TRAI

TRAIN_9244\01\يا سبحان الله عنجد ام حلوة كتير وعظيمة كتير ويارب بنات هاد الجيل يتعلموا منا حت لو ربع هالشغلات الرائعة 
TRAIN_9245\01\بتعرفي تعبوني كتير
TRAIN_9246\01\سالوا الصعيدي انت كيف بتعرفني
TRAIN_9247\01\الاقيك هان
TRAIN_9248\01\ليش الدكتور احسن من الله الي فرض الحجاب
TRAIN_9249\01\معناها ما لازم تخافوا و لازم تبدو الشغل من اول ثلاثي كل يوم
TRAIN_9250\01\الحمد لله و انت بخير
TRAIN_9251\01\هي اقل من المتصرف الاداري
TRAIN_9252\01\انا ما وجعونيش تورموا وبس
TRAIN_9253\01\نشكر الله سهام انا اتصلت بنرجس هلا
TRAIN_9254\01\اه اقولك تصبحي على خير
TRAIN_9255\01\فهمت
TRAIN_9256\01\يا ريت نصلح القطار  بس مش على حساب الفقرا الي بستخدموه
TRAIN_9257\01\راحو عالساعة وحدة
TRAIN_9258\01\الاولى صعبة المغربية و التونسية شوية
TRAIN_9259\01\بلد بنت كلب  حقودة وعمى قلوب اهلها 
TRAIN_9260\01\بيخسؤ سمعته منشان شغلة متل هي
TRAIN_9261\01\لا ما اعطيها شرائحي
TRAIN_9262\01\وبعدين يعني انا رح انخطف  ولا رح انخطف 
TRAIN_9263\01\اعطاك كف حتى وجهك لف
TRAIN_9264\01\هاذ الي كان بدي اياه 
TRAIN_9265\01\بما انكم ما 

In [116]:
cat data/train/msa_formatted_.txt

TRAIN_0000\01\ربما تكون مسالة السلاح الكيمياوي مفتعلة لاتخاذها ذريعة كما حدث في العراق
TRAIN_0001\01\تتطلب قليلا من المعالجة
TRAIN_0002\01\احسبي فرق الساعات بين اذان العشاء و اذان الصبح ثم اقسمي المجموع على ثلاثة و ستحصلين على النتيجة
TRAIN_0003\01\نعم انه جيد
TRAIN_0004\01\و بتوفيق الله وجدت على غلافه عنوان ناشر الكتاب  فراسلته لاطلب المزيد فارسلوا لي ما طلبت
TRAIN_0005\01\ اطلب منه اذا استطاع ان يمر ان ياخذ لي موعدا عنده   
TRAIN_0006\01\اقصد احترم نفسك هذا لا شيء
TRAIN_0007\01\لو انني على سبيل المثال اجد منصبا لدى الدولة و اعمل هناك  استطيع ان احصل على راتب من هنا و من هناك
TRAIN_0008\01\لا اريد ان العب
TRAIN_0009\01\الان سيبدا ترديد كلمات اقرا  احفظ  اكتب  والبقية
TRAIN_010\01\كانك تعلم شخصا كيف يتعلم شيئا
TRAIN_011\01\ركزي تفكيرك و اتركي اللعب جانبا و دعي عنك هذه الحكايات و صفي ذهنك
TRAIN_012\01\حقا
TRAIN_013\01\امسكت زينو اليوم
TRAIN_014\01\و هو كذلك
TRAIN_015\01\ شخص مثل هذا يفكر في مصلحته الخاصة قبل مصلحة الطالب   
TRAIN_016\01\لقد قدمت ملفا للحصول على عقد
TRAIN_017\01\نعم
TRAI

TRAIN_3074\01\ما هو اكثر شئ يؤلمك واثر على حياتك 
TRAIN_3075\01\و لكن انتبهوا العفريت ناصح كل شخص سياره واحده فقط
TRAIN_3076\01\ما هو موضوع اليوم
TRAIN_3077\01\و لكن الايام تمر كلها مثل بعضها و لا يوجد جديد
TRAIN_3078\01\كريستيان دمت ضاحكا ومسرورا
TRAIN_3079\01\الذى يخاف من الحرب يخاف لانه لديه شئ مهم يخسره
TRAIN_3080\01\اول شيء اضحكتني شكرا
TRAIN_3081\01\عما تتحدثين  ماذا قلت لها
TRAIN_3082\01\ستنامين
TRAIN_3083\01\هذا شيء غير معقول
TRAIN_3084\01\سالته: متى سنناقش المذكرة يا استاذ قال لي : اصبر قليلا
TRAIN_3085\01\ عندما كان كذلك طرحته ارضا و هربت تركض بقي اولادها فاصطادهم اخرون   
TRAIN_3086\01\هنا
TRAIN_3087\01\نعم
TRAIN_3088\01\ حوالي الخمسينات تقريبا   
TRAIN_3089\01\فشل مرة اخرى
TRAIN_3090\01\هل تتصورى ميف كنت مرتاح من شقاوتك صديقى تحدث معى فى الهاتف و يبعت لكى السلام ايتها المشاغبه
TRAIN_3091\01\ما بك هل مازلت صائمة
TRAIN_3092\01\و هذه الراحة النفسية جعلتني مبدعا فيها
TRAIN_3093\01\هذا هو الشيء الاخير الصواب الذي يفعله الفتيان
TRAIN_3094\01\لا اعلم متى تعود امي
TRAIN_3095\01\دائ

TRAIN_7659\01\ اؤمن به لا العكس لكن حب لا تاتي معه الرجولة ليس له اي قيمة   
TRAIN_7660\01\انهم  جدا محظوظون   مازالوا يلعبون بالعجين و القريصات و الخشيبات
TRAIN_7661\01\مثل هدى ما شاء الله
TRAIN_7662\01\انها اول مرة التي  ارى فيها وجهك بهذا الشكل
TRAIN_7663\01\فليحفظنا الله
TRAIN_7664\01\لا انا ادرس في المكتبة
TRAIN_7665\01\خيرا ان شاء الله و هل مر الامر بسلام
TRAIN_7666\01\انت من المفترض انك ذهبت الى الرجل الذى فى الشركه و شرحت له الموضوع
TRAIN_7667\01\فعلا  لم يتم حتى ملا المحافظ
TRAIN_7668\01\شكرا على هذا الموضوع الرائع و من عاداتى 
TRAIN_7669\01\ليست بالبيت يا زينب
TRAIN_7670\01\و ايضا طريقه النقل التى ننقل بها تعتبر سرقه لاننا نقول منقول بدون ان نكتب المصدر الذى تم النقل منه
TRAIN_7671\01\نعم
TRAIN_7672\01\اقترح انت مراد
TRAIN_7673\01\اف انه تقريبا مصنوع من الماء
TRAIN_7674\01\هيا اين ذهب الخيط
TRAIN_7675\01\الاهم هو الاستفادة
TRAIN_7676\01\ينادونك في كل مكان تارة  و احيانا انزلي الى الاسفل
TRAIN_7677\01\ غير الموضوع   
TRAIN_7678\01\ عندما يتعب ينام فيها لماذا لان هذا السيد  عند

## Build Vocabulary 

Sentence Piece Google Colab
https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb

In [26]:
len(data_English_MSA)

10001

In [27]:
"""
Create a text file with all the MSA vocab available for SentencePiece to create a library 
"""

text_file_en = open("data/english_data.txt", "wt")
text_file_msa = open("data/arabic_data.txt", "wt")

MSA_text = ""
EN_text = ""

for line in data_English_MSA:
        english = line[0]
        english_words = english.split(" ")
        for count, word in enumerate(english_words):
            text_file_en.write(word)
        text_file_en.write("\n")
        
        arabic = line[1]
        arabic_words = arabic.split(" ")
        for count, word in enumerate(arabic_words):
            text_file_msa.write(word)
        text_file_msa.write("\n")


text_file_en.close()
text_file_msa.close()

In [28]:
spm.SentencePieceTrainer.train('--input=data/arabic_data.txt --model_prefix=data/msa --vocab_size=2000')
spm.SentencePieceTrainer.train('--input=data/english_data.txt --model_prefix=data/en --vocab_size=2000')

True

In [29]:
sp = spm.SentencePieceProcessor()
sp.load('data/en.model')

True

In [30]:
ls

Env_Setup_Instructions.txt  data/
Model_1.ipynb               transformers/
README.md


In [31]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁', 'is', '▁', 'a', '▁', 't', 'est']
[89, 83, 12, 83, 8, 83, 6, 309]


## Spacy Tokenizer

In [32]:
ls data/

AD_NMT-master/
AD_NMT-master.zip
PADIC.xml
UNv1.0.ar-en.tar.gz.00
UNv1.0.ar-en.tar.gz.01
ar_formatted_test.txt
ar_formatted_train.txt
ar_formatted_validation.txt
arabic-to-english-translation-sentences.zip
arabic_data.txt
en.model
en.vocab
en_formatted_.txt
en_formatted_test.txt
en_formatted_train.txt
english_data.txt
lav_formatted_.txt
mag_formatted_.txt
msa.model
msa.vocab
msa_formatted_.txt
test/
train/
validation/


In [34]:
text_file_en = open("data/en.vocab", "r")
text_file_ar = open("data/msa.vocab", "r")

en_vocab_list = []
ar_vocab_list = []
for line in text_file_en.readlines():
    en_vocab_list.append(line.split("\t")[0])

for line in text_file_ar.readlines():
    print(line)
    print(line.split("\t")[0])
    print("---------")
    ar_vocab_list.append(line.split("\t")[0])

<unk>	0

<unk>
---------
<s>	0

<s>
---------
</s>	0

</s>
---------
ا	-3.13275

ا
---------
▁	-3.1368

▁
---------
ت	-3.45305

ت
---------
ي	-3.56965

ي
---------
ال	-3.57586

ال
---------
؟	-3.73331

؟
---------
ب	-3.99997

ب
---------
م	-4.04006

م
---------
ك	-4.05122

ك
---------
ة	-4.19093

ة
---------
ل	-4.22863

ل
---------
ه	-4.35769

ه
---------
ن	-4.60958

ن
---------
في	-4.72085

في
---------
و	-4.78685

و
---------
ع	-4.84808

ع
---------
▁ا	-4.85412

▁ا
---------
توم	-4.92814

توم
---------
من	-5.00118

من
---------
ها	-5.08404

ها
---------
ف	-5.09055

ف
---------
ان	-5.12223

ان
---------
د	-5.12656

د
---------
ر	-5.13282

ر
---------
ق	-5.13918

ق
---------
نا	-5.25225

نا
---------
ح	-5.31822

ح
---------
ني	-5.32711

ني
---------
ج	-5.3788

ج
---------
ط	-5.38223

ط
---------
ما	-5.40789

ما
---------
ص	-5.43782

ص
---------
خ	-5.44452

خ
---------
ش	-5.55425

ش
---------
ين	-5.66536

ين
---------
ز	-5.68319

ز
---------
ذلك	-5.6844

ذلك
---------
على	-5.69837

على


ساعد
---------
اذا	-7.87932

اذا
---------
قتل	-7.88073

قتل
---------
اكثرمن	-7.88104

اكثرمن
---------
▁لمت	-7.88486

▁لمت
---------
يستطيع	-7.88793

يستطيع
---------
عي	-7.88981

عي
---------
دفع	-7.89229

دفع
---------
ايضا	-7.89258

ايضا
---------
صوت	-7.89723

صوت
---------
شتري	-7.89884

شتري
---------
طعام	-7.8991

طعام
---------
قرب	-7.90052

قرب
---------
ثلاثة	-7.90253

ثلاثة
---------
حول	-7.90307

حول
---------
▁ليسلدي	-7.91079

▁ليسلدي
---------
جيد	-7.91455

جيد
---------
كبيرة	-7.91666

كبيرة
---------
▁يمكن	-7.91705

▁يمكن
---------
ارة	-7.91706

ارة
---------
طالب	-7.91951

طالب
---------
الصين	-7.92164

الصين
---------
حضر	-7.92451

حضر
---------
تزوج	-7.92663

تزوج
---------
▁انهات	-7.92672

▁انهات
---------
در	-7.93087

در
---------
للغاية	-7.93122

للغاية
---------
ترجم	-7.93131

ترجم
---------
رغب	-7.93425

رغب
---------
جيدا	-7.93598

جيدا
---------
مني	-7.9362

مني
---------
السياح	-7.93655

السياح
---------
كره	-7.93662

كره
---------
لتوم	-7.93732

لتوم
-----

---------
لميكن	-8.4825

لميكن
---------
شراء	-8.48454

شراء
---------
سد	-8.48677

سد
---------
حبه	-8.48974

حبه
---------
جاد	-8.49172

جاد
---------
اعدا	-8.49272

اعدا
---------
طار	-8.49454

طار
---------
▁اعرف	-8.49893

▁اعرف
---------
بريد	-8.49924

بريد
---------
▁عليانا	-8.49993

▁عليانا
---------
▁دع	-8.49995

▁دع
---------
تماما	-8.50019

تماما
---------
قاب	-8.50749

قاب
---------
خر	-8.50892

خر
---------
اتال	-8.50971

اتال
---------
ركب	-8.51208

ركب
---------
لطف	-8.51431

لطف
---------
نفس	-8.51493

نفس
---------
اربع	-8.51544

اربع
---------
الطبيب	-8.51721

الطبيب
---------
طعم	-8.51733

طعم
---------
تحتاج	-8.5237

تحتاج
---------
اليه	-8.5258

اليه
---------
لوسمحت	-8.52705

لوسمحت
---------
▁اعطني	-8.52705

▁اعطني
---------
الحديقة	-8.52706

الحديقة
---------
دراجة	-8.52706

دراجة
---------
▁دعونان	-8.52707

▁دعونان
---------
اللغةالانجليزية	-8.52709

اللغةالانجليزية
---------
كيوتو	-8.52711

كيوتو
---------
قاموس	-8.52714

قاموس
---------
دراسة	-8.52717

دراسة
-

---------
النار	-8.90973

النار
---------
دور	-8.91194

دور
---------
سابق	-8.91358

سابق
---------
▁اعتقد	-8.91638

▁اعتقد
---------
مدينة	-8.91827

مدينة
---------
▁اتذكر	-8.91922

▁اتذكر
---------
انجليزية	-8.92075

انجليزية
---------
لاجل	-8.95027

لاجل
---------
منزلك	-8.95995

منزلك
---------
تقال	-8.96005

تقال
---------
اذاكنت	-8.9628

اذاكنت
---------
فرنسي	-8.96822

فرنسي
---------
لاتستطيع	-8.96849

لاتستطيع
---------
اشتر	-8.97305

اشتر
---------
الخروج	-8.97558

الخروج
---------
اسمي	-8.98495

اسمي
---------
اخيرا	-8.98519

اخيرا
---------
جائع	-8.98618

جائع
---------
العديدمنال	-8.98818

العديدمنال
---------
▁ذهبت	-8.98879

▁ذهبت
---------
نظيف	-8.98931

نظيف
---------
2	-8.98932

2
---------
عيدميلاد	-8.98932

عيدميلاد
---------
ربطةعنق	-8.98932

ربطةعنق
---------
بالنسبةلي	-8.98932

بالنسبةلي
---------
اقتصاد	-8.98932

اقتصاد
---------
فرنسا	-8.98934

فرنسا
---------
التقرير	-8.98935

التقرير
---------
19	-8.98938

19
---------
الالمانية	-8.98939

الالمانية
---------
ق

---------
امزح	-9.6431

امزح
---------
▁ماذاتعتقد	-9.6468

▁ماذاتعتقد
---------
اقلام	-9.64899

اقلام
---------
طلاقا	-9.65214

طلاقا
---------
▁تبداال	-9.66137

▁تبداال
---------
متىستعود	-9.67627

متىستعود
---------
اعلمانه	-9.68375

اعلمانه
---------
عتقدذلك	-9.72688

عتقدذلك
---------
تجاري	-9.77029

تجاري
---------
▁كمعمر	-9.78563

▁كمعمر
---------
▁انهيتكلمال	-9.79073

▁انهيتكلمال
---------
تلفاز	-9.81427

تلفاز
---------
انتخبرني	-9.84926

انتخبرني
---------
دقائق	-9.87352

دقائق
---------
9	-9.87368

9
---------
الاعتمادعل	-9.87369

الاعتمادعل
---------
خبرنيب	-9.8737

خبرنيب
---------
الجزءال	-9.87383

الجزءال
---------
باستمرار	-9.87383

باستمرار
---------
▁تظاهر	-9.87384

▁تظاهر
---------
▁لااحداعرفهي	-9.87384

▁لااحداعرفهي
---------
اعتمدعليك	-9.87384

اعتمدعليك
---------
حديقةالحيوان	-9.87384

حديقةالحيوان
---------
فيانجلترا	-9.87384

فيانجلترا
---------
مليئةبال	-9.87384

مليئةبال
---------
الجائزة	-9.87384

الجائزة
---------
ثمنالتذاكر	-9.87384

ثمنالتذاكر
---------
حاف

اعضاءالفريقب	-10.7072

اعضاءالفريقب
---------
اعمللحسابشركةفيا	-10.7072

اعمللحسابشركةفيا
---------
اكلطعامالفطور	-10.7072

اكلطعامالفطور
---------
الةتصوير	-10.7072

الةتصوير
---------
الرغممنفقره	-10.7072

الرغممنفقره
---------
الطائرةالمقاتلةا	-10.7072

الطائرةالمقاتلةا
---------
الطرقتؤديالى	-10.7072

الطرقتؤديالى
---------
ظ	-10.8037

ظ
---------


In [35]:
ar_vocab_list[0:200]

['<unk>',
 '<s>',
 '</s>',
 'ا',
 '▁',
 'ت',
 'ي',
 'ال',
 '؟',
 'ب',
 'م',
 'ك',
 'ة',
 'ل',
 'ه',
 'ن',
 'في',
 'و',
 'ع',
 '▁ا',
 'توم',
 'من',
 'ها',
 'ف',
 'ان',
 'د',
 'ر',
 'ق',
 'نا',
 'ح',
 'ني',
 'ج',
 'ط',
 'ما',
 'ص',
 'خ',
 'ش',
 'ين',
 'ز',
 'ذلك',
 'على',
 '▁هل',
 'لا',
 'ى',
 'الى',
 'س',
 'انت',
 '!',
 'ون',
 'ية',
 '▁انا',
 'عن',
 'ض',
 'هذا',
 'هنا',
 'لي',
 'الم',
 'غ',
 'ار',
 'ست',
 'ئ',
 'اء',
 'اني',
 'فيال',
 'ته',
 'بال',
 'كل',
 'وا',
 '▁سا',
 '▁لقد',
 'عمل',
 'الان',
 'هم',
 'ث',
 'ذ',
 'بعد',
 'ذهب',
 'انا',
 'ات',
 'هو',
 '▁توم',
 '▁انه',
 '▁انت',
 'تك',
 'عد',
 'منال',
 'جدا',
 '▁هذا',
 'را',
 'مع',
 'كان',
 'سر',
 'ام',
 '▁ال',
 'ور',
 'لل',
 'تحدث',
 'سي',
 'ماري',
 'هي',
 'لك',
 'هذهال',
 'اس',
 '▁كان',
 'فعل',
 '▁لات',
 'كتب',
 'اد',
 'قل',
 'اليوم',
 'علىال',
 'هناك',
 'ابي',
 'سا',
 '"',
 'قد',
 '▁ما',
 'ير',
 'ري',
 'اكل',
 '▁انها',
 'مس',
 '▁اين',
 'احد',
 'اع',
 'غدا',
 'قبل',
 'هذاال',
 '▁هلت',
 'حقا',
 'عرف',
 '▁ان',
 '▁لن',
 'اف',
 'انه',
 '▁م

In [36]:
text_file_en = open("data/en.vocab", "r")
text_file_ar = open("data/msa.vocab", "r")

line = text_file_en.readlines()

In [37]:
en_vocab = Vocab(strings=en_vocab_list)
spacy_en_tokenizer = Tokenizer(en_vocab)

ar_vocab = Vocab(strings=ar_vocab_list)
spacy_msa_tokenizer = Tokenizer(ar_vocab)

## TF Tokenizer

https://huggingface.co/transformers/model_doc/t5.html#t5tokenizer

In [38]:
ls data/

AD_NMT-master/
AD_NMT-master.zip
PADIC.xml
UNv1.0.ar-en.tar.gz.00
UNv1.0.ar-en.tar.gz.01
ar_formatted_test.txt
ar_formatted_train.txt
ar_formatted_validation.txt
arabic-to-english-translation-sentences.zip
arabic_data.txt
en.model
en.vocab
en_formatted_.txt
en_formatted_test.txt
en_formatted_train.txt
english_data.txt
lav_formatted_.txt
mag_formatted_.txt
msa.model
msa.vocab
msa_formatted_.txt
test/
train/
validation/


In [39]:
msa_tokenizer = T5Tokenizer("data/msa.model")
en_tokenizer = T5Tokenizer("data/en.model")

In [40]:
input_ids = msa_tokenizer.encode('هل بامكاني الدخول؟ </s>', return_tensors='pt')

In [41]:
input_ids

tensor([[ 41,   4, 330,   6,  93, 964,   8,   4,   2]])

In [42]:
#decode to make sure you can go back and forth between the encoding properly 
#@raef can you verify the Arabic?

In [50]:
ls data/train

en_formatted_train.txt


## Pytorch Data Set and Data Loader

https://github.com/google-research/text-to-text-transfer-transformer

pytorch dataset: https://pytorch.org/text/_modules/torchtext/datasets/translation.html

pytorch dataset documentation: https://torchtext.readthedocs.io/en/latest/datasets.html#iwslt

example dataset: https://iwslt2010.fbk.eu/node/32/

Field API: https://pytorch.org/text/data.html#torchtext.data.Field
Field Source: https://pytorch.org/text/_modules/torchtext/data/field.html#Field

https://pytorch.org/text/_modules/torchtext/data/utils.html#get_tokenizer
    
we need to pass our tokenizer as a function     

In [44]:
SRC = Field(tokenize = spacy_en_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

TRG = Field(tokenize = spacy_msa_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

In [51]:
dataset = TranslationDataset(path='data/', exts=('en_formatted_.txt', 'msa_formatted_.txt'), fields=(SRC, TRG))

In [46]:
SRC.build_vocab(dataset)
TRG.build_vocab(dataset)

In [86]:
ex = dataset[0]

In [85]:
print(ex.src) 
print(ex.trg)

_0000\01\Tom was also there
_0000\01\يارب لا تدع اهدافهم تصيب حتى العارضة


In [68]:
ls data/train

en_formatted_train.txt


In [53]:
#train, validation and test are probably wrong.  
#https://github.com/pytorch/text/blob/master/torchtext/datasets/translation.py

'''
Arguments:
            exts: A tuple containing the extension to path for each language.
            fields: A tuple containing the fields that will be used for data
                in each language.
            path (str): Common prefix of the splits' file paths, or None to use
                the result of cls.download(root).
            root: Root dataset storage directory. Default is '.data'.
            train: The prefix of the train data. Default: 'train'.
            validation: The prefix of the validation data. Default: 'val'.
            test: The prefix of the test data. Default: 'test'.
            Remaining keyword arguments: Passed to the splits method of
                Dataset.

'''


train_data, valid_data, test_data = dataset.splits(path= 'data/', exts=('/en_formatted_.txt', '/msa_formatted_.txt'),
                                                    fields = (SRC, TRG))

FileNotFoundError: [Errno 2] No such file or directory: 'data/train/en_formatted_.txt'

In [ ]:
BATCH_SIZE =32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

## Model Training

https://huggingface.co/transformers/model_doc/t5.html#training

In [ ]:
configuration = T5Config()

In [ ]:
model = T5Model.from_pretrained('t5-base')

In [ ]:
for i, batch in enumerate(train_iterator)

## Model Inference

https://github.com/huggingface/transformers/blob/master/examples/translation/t5/evaluate_wmt.py